In [12]:
# Importing the required notebooks
%run DifferentialEvolution.ipynb
%run IslandMethod.ipynb

In [15]:
import os
import sys
import numpy as np
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import Row

from bbobbenchmarks import fgeneric
from bbobbenchmarks import bbobbenchmarks as bn

# Start spark
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('IPDE') \
    .getOrCreate()
sc = spark.sparkContext
    
class Evaluation:

    # Setting the different parameters to evaluate
    _test = {'CR' : '_testCR', 'F' : '_testF'}
    
    
    def __init__(self, title):
        # Setting the CSV title
        self.title = title
        
        # Deleting file if it already exists
        self.filename = "outputs/" + title + ".csv"
        if(os.path.isfile(self.filename)):
            os.remove(self.filename)
            
        # Create the file or open
        #self.file = open(filename, "a")
        
        # Setting header for the CSV
        output_header = "Function,Instance,Dimensions,NP,CR,F,MuSt,CRSt,Local Iterations,"
        output_header += "Workers,Evaluations,Migrations,Global Time,Chromosome,Target,"
        output_header += "Score,Stop Reason"
        self.print_output(output_header)
        
        
    def run(self, test = 'CR', workers = 4, localIt = 2):
        # Setting the functions to test, dimension of individuals and its instan ces
        dimensions = [4]
        function_ids = [6]
        instances = range(1, 11)

        # Running all functions
        f = fgeneric.LoggingFunction('tmp')
        values = np.arange(0.1, 1.0, 0.05) if test == 'CR' else np.arange(0.1, 2.0, 0.1)
        for value in values:
            for dim in dimensions:
                for fun_id in function_ids:
                    for iinstance in instances:
                        # Setting new function to evaluate
                        f.setfun(*bn.instantiate(fun_id, iinstance=iinstance))
                        print("\nFunction:", fun_id, "Instance:", iinstance)
                        
                        # Evaluation of the functions
                        diffEvoTest = getattr(self, self._test[test])
                        diffEvo = diffEvoTest(f.evalfun, goal = f.ftarget, dim = dim, value = value)
                        island = IslandMethod(diffEvo = diffEvo, workers = workers, localIt = localIt)
                        evaluations = island.execute(sc)

                        # Storing the results obtained
                        elements = [fun_id, iinstance, dim, diffEvo.np, diffEvo.cr, diffEvo.f,
                                    diffEvo.MuSt, diffEvo.CRSt, localIt, workers, evaluations, 
                                    island.migrationCount, island.globalTime, island.get_best_chromosome(), 
                                    f.ftarget, island.get_best_score(), island.stoppingReason]
                        output_data = ",".join(str(e) for e in elements)

                        # Printing output
                        self.print_output(output_data)
        

    def _testCR(self, func, goal, dim, value):
        return DifferentialEvolution(func, goal = goal, d = dim, cr = value)
    
    
    def _testF(self, func, goal, dim, value):
        return DifferentialEvolution(func, goal = goal, d = dim, f = value)
    
        
    def print_output(self, output):
        print(output, file = open(self.filename, "a"))

In [7]:
test = Evaluation("CR-test")
test.run()

[Errno 17] File exists: 'tmp'

Number of Evaluations 20

Initial partition structure: [[(0, (array([-3.61624135,  0.25200261,  1.82894383, -3.52203906]), 132.63721618722079)), (1, (array([ 4.83339615, -3.73665403,  4.83267932, -4.69326061]), 181.81409952985558))], [(2, (array([-1.93585046, -4.67486356, -3.93585842,  3.88781699]), 110.81015318239966)), (3, (array([ 4.20013899, -3.46521971,  2.80850977, -2.42322657]), 131.78816328365923))], [(4, (array([-2.15380717,  4.55496892, -1.26034326,  4.12108199]), 123.00035542364091)), (5, (array([ 3.2900197 , -4.61533438,  1.55621778, -2.78705205]), 128.08218692526955))], [(6, (array([ 1.1422667,  3.882018 , -4.3819872, -3.3202635]), 146.56918616708978)), (7, (array([-2.23857593,  1.0838928 ,  3.69515618,  4.26335303]), 115.69794907657767)), (8, (array([ 1.23117122,  4.75289637, -0.19788117, -0.31900887]), 120.68038346469903)), (9, (array([-2.19560299,  4.40894719, -1.50421693,  0.95541423]), 118.0037707977551))], [(10, (array([-3.35357767,  4.


Best chromosome ->: [(7, (array([ 0.25279984, -1.15680218, -0.72399957,  1.92640148]), 79.480000000007152))]

Values after island: [(7, (array([ 0.25279984, -1.15680218, -0.72399957,  1.92640148]), 79.480000000007152)), (11, (array([ 0.25280085, -1.15680195, -0.7240005 ,  1.92639819]), 79.480000000008033)), (3, (array([ 0.25280022, -1.15680195, -0.72400076,  1.9264026 ]), 79.480000000011202)), (19, (array([ 0.25280023, -1.15680233, -0.72400011,  1.92639632]), 79.480000000019047)), (5, (array([ 0.25280662, -1.15680614, -0.72399614,  1.92639666]), 79.480000000107637)), (15, (array([ 0.25280018, -1.15678654, -0.72399922,  1.92639965]), 79.480000000181946)), (10, (array([ 0.25281461, -1.15679646, -0.72400617,  1.92639739]), 79.480000000270977)), (16, (array([ 0.25281464, -1.15680576, -0.72399033,  1.92639995]), 79.480000000341164)), (2, (array([ 0.25279497, -1.15680457, -0.7240096 ,  1.92641764]), 79.480000000449536)), (9, (array([ 0.25277804, -1.15680153, -0.72400068,  1.92639973]), 79.4


Best chromosome ->: [(9, (array([-3.89863405, -2.89050907, -3.80293099,  3.90550749]), 394.48000035718258))]

Values after island: [(9, (array([-3.89863405, -2.89050907, -3.80293099,  3.90550749]), 394.48000035718258)), (1, (array([-3.89870536, -2.89038273, -3.80186634,  3.90601767]), 394.48000055279368)), (5, (array([-3.89775429, -2.89040669, -3.80183331,  3.90585588]), 394.48000080359952)), (17, (array([-3.89729781, -2.89041286, -3.80178376,  3.90536534]), 394.48000164980704)), (6, (array([-3.90078515, -2.89022875, -3.80391695,  3.9061796 ]), 394.48000835537556)), (3, (array([-3.89663306, -2.88892691, -3.80344603,  3.90707223]), 394.4800085536757)), (13, (array([-3.89647829, -2.89042724, -3.7999543 ,  3.90554977]), 394.48000967770093)), (14, (array([-3.9007933 , -2.89050157, -3.80384707,  3.90381407]), 394.48001102175101)), (10, (array([-3.9007933 , -2.89024497, -3.80465298,  3.90345759]), 394.48001541775517)), (11, (array([-3.89542718, -2.89161828, -3.80009608,  3.90641152]), 394.4


Best chromosome ->: [(15, (array([ 1.00190505,  0.49581456,  3.55609955,  2.38642337]), -209.53832436730406))]

Values after island: [(15, (array([ 1.00190505,  0.49581456,  3.55609955,  2.38642337]), -209.53832436730406)), (7, (array([ 1.03748934,  0.47831363,  3.55518384,  2.38728216]), -208.98203435315148)), (11, (array([ 0.76918466,  0.43643069,  3.54852418,  2.38572429]), -208.8603475647088)), (3, (array([ 0.82849181,  0.48487648,  3.55499846,  2.38541392]), -208.71639511414128)), (19, (array([ 1.1134061 ,  0.4041335 ,  3.54439337,  2.38684524]), -208.56490351601798)), (6, (array([ 1.09624994,  0.78183147,  3.49162015,  2.38132929]), -134.07557684232125)), (18, (array([ 1.10421241,  0.70656181,  3.43523861,  2.38765746]), -53.018066871119458)), (14, (array([ 1.47095846,  0.59341265,  3.57416203,  2.40436382]), 90.302102597666021)), (9, (array([ 1.37711429, -1.54626946,  3.54577985,  2.39099757]), 221.92825580114214)), (13, (array([ 1.52375308, -1.6309525 ,  3.52413488,  2.3921183


Values after island 2: [(5, (array([ 1.20688916,  0.44805368,  3.55439874,  2.38639993]), -209.87999959424567)), (3, (array([ 1.20626381,  0.44801253,  3.55439877,  2.38639987]), -209.87999915047646)), (7, (array([ 1.20747667,  0.44796837,  3.55441082,  2.38639987]), -209.87999882555275)), (18, (array([ 1.20737729,  0.44795481,  3.55439001,  2.38640043]), -209.87999852439802)), (10, (array([ 1.20766218,  0.44798368,  3.55438956,  2.38639949]), -209.87999833989835)), (6, (array([ 1.20750007,  0.4480804 ,  3.55439034,  2.38639977]), -209.87999820509395)), (9, (array([ 1.20653198,  0.44810284,  3.5543955 ,  2.38640004]), -209.87999817773502)), (19, (array([ 1.20626381,  0.44801509,  3.55441235,  2.38640056]), -209.87999746815979)), (17, (array([ 1.20738744,  0.44815431,  3.55441034,  2.38639984]), -209.87999661763345)), (11, (array([ 1.20607721,  0.44799008,  3.55440512,  2.38639871]), -209.87999661452946)), (15, (array([ 1.20758863,  0.44796205,  3.55439621,  2.38639806]), -209.87999579

...


Values after island 2: [(5, (array([ 1.42078871,  1.40184059,  3.87022938,  1.30752847]), 35.6612774254545)), (1, (array([ 0.60783735,  0.71978017,  3.62456756,  0.63207448]), 35.976914604073507)), (19, (array([-0.36284363, -0.25755053,  3.04153571,  0.09964001]), 37.470127630850044)), (9, (array([-0.60252197,  0.45166931,  2.45739234,  1.52280393]), 37.695501854640483)), (10, (array([ 0.78175299,  0.7558463 ,  3.78061332,  0.58961588]), 37.771763071883072)), (4, (array([-1.09964861, -1.03335057,  2.12089665,  0.24553795]), 38.053660800356695)), (16, (array([-0.35806966,  0.08163098,  3.37509642,  0.32965113]), 38.190079268750921)), (17, (array([-2.22114952, -1.10749842,  1.92493735, -0.0044121 ]), 38.199191617479833)), (11, (array([-1.89498857, -1.0697198 ,  2.06075104,  0.02582453]), 38.199191617479833)), (2, (array([-0.45729821, -0.32957657,  2.04074048,  1.34510304]), 38.348818350422107)), (13, (array([-1.9574409 , -1.0539341 ,  2.29727422, -0.26549714]), 39.28582926244399)), (14,


Best chromosome ->: [(13, (array([ 1.58725608,  1.23242106,  3.87022938,  1.36038814]), 35.474829633876311))]

Values after island: [(13, (array([ 1.58725608,  1.23242106,  3.87022938,  1.36038814]), 35.474829633876311)), (14, (array([ 0.60783735,  0.71978017,  3.62456756,  0.63207448]), 35.976914604073507)), (1, (array([ 1.15206824,  0.71941493,  3.78061332,  0.58961588]), 36.042922074311718)), (6, (array([ 2.8002789 ,  1.30459478,  4.47433247,  1.01350544]), 36.837842601372145)), (9, (array([ 3.82477047,  2.14707728,  5.21968031,  1.21308132]), 37.153811221115227)), (2, (array([ 0.06470501,  0.08163098,  3.28322226,  0.32965113]), 37.171134660147928)), (4, (array([-1.09964861, -0.68435734,  2.38232117,  0.29944801]), 37.246173501576571)), (17, (array([-0.16179859, -0.4033409 ,  2.65531239,  0.52615037]), 37.441561125487986)), (16, (array([-0.36284363, -0.25755053,  3.04153571,  0.09964001]), 37.470127630850044)), (0, (array([-0.45729821,  0.42076656,  2.59008055,  1.34510304]), 37.6


Values after island 2: [(2, (array([ 1.58725608,  1.23242106,  3.87022938,  1.36038814]), 35.474829633876311)), (16, (array([ 0.81297251,  0.71978017,  3.67875099,  0.63207448]), 35.852562712320449)), (9, (array([ 1.15206824,  0.71941493,  3.78061332,  0.58961588]), 36.042922074311718)), (0, (array([ 0.06470501,  0.38934264,  3.28322226,  0.66319798]), 36.102948798243517)), (14, (array([ 1.79874106,  1.66003435,  4.47433247,  1.01350544]), 36.207650835231625)), (3, (array([ 3.82477047,  2.14707728,  5.21968031,  1.21308132]), 37.153811221115227)), (8, (array([-1.09964861, -0.68435734,  2.38232117,  0.29944801]), 37.246173501576571)), (5, (array([-0.16179859,  0.7512474 ,  2.75779983,  1.82226721]), 37.328856912607485)), (12, (array([-0.36284363, -0.25755053,  3.04153571,  0.09964001]), 37.470127630850044)), (17, (array([-0.45729821,  0.42076656,  2.59008055,  1.34510304]), 37.695501854640483)), (7, (array([-0.60252197,  0.45166931,  2.45739234,  1.52280393]), 37.695501854640483)), (4,


Best chromosome ->: [(11, (array([ 1.58725608,  1.23242106,  3.87022938,  1.36038814]), 35.474829633876311))]

Values after island: [(11, (array([ 1.58725608,  1.23242106,  3.87022938,  1.36038814]), 35.474829633876311)), (12, (array([ 0.739954  ,  0.71978017,  3.67875099,  0.63207448]), 35.792337685307778)), (7, (array([ 1.05747567,  0.71941493,  3.78061332,  0.58961588]), 35.926034622510777)), (10, (array([ 0.07468285,  0.38934264,  3.28322226,  0.66319798]), 36.018384040701619)), (1, (array([ 1.79874106,  1.66003435,  4.47433247,  1.01350544]), 36.207650835231625)), (14, (array([ 0.01064466,  0.2584747 ,  2.59008055,  1.34510304]), 36.854838608735086)), (0, (array([ 0.02894568,  0.45166931,  2.67290592,  1.52280393]), 37.048769135862656)), (18, (array([-0.16179859,  0.84071859,  2.71990752,  1.82226721]), 37.048769135862656)), (19, (array([ 3.82477047,  2.14707728,  5.21968031,  1.21308132]), 37.153811221115227)), (15, (array([-1.09964861,  0.13627305,  2.38232117,  1.00397135]), 3

KeyboardInterrupt: 

In [ ]:
test = Evaluation("F-test")
test.run(test = 'F')

[Errno 17] File exists: 'tmp'

Function: 6 Instance: 1

Number of Evaluations 80

Initial partition structure: [[(0, (array([ 2.77582013, -1.82081837,  0.68940294,  0.53261362]), 68262.109274804097)), (1, (array([-4.42244635,  1.08725399,  3.55720183, -2.11296502]), 205496.80848834236)), (2, (array([-3.3484585 , -3.82329354, -0.94169765,  4.31449473]), 116.80408782417683)), (3, (array([-2.71898339,  3.62055198, -4.75000365,  4.3213381 ]), 38564.852797267566)), (4, (array([-2.92501004, -1.50958814,  4.29788157, -2.62266314]), 296124.30877408542)), (5, (array([-2.70030797, -3.35035412, -3.61950936,  1.07146618]), 182.02801491661867)), (6, (array([ 0.36854635,  1.24112858,  4.33485301,  1.7382845 ]), 53176.004187324324)), (7, (array([ 1.12949342,  3.88777267,  4.17054597, -4.17389195]), 570943.81192426186)), (8, (array([ 0.39754782,  0.60205452,  4.02311359,  2.41947828]), 19727.630572802947)), (9, (array([ 2.5021644 ,  0.04914991, -0.43753697,  4.74021045]), 1811.2548998350383))], [(10, 


Best chromosome ->: [(9, (array([ 1.80899394,  0.83971264,  1.44192858,  3.22061377]), 44.147500245419437))]

Migration number: 5

Number of Evaluations 880
Number of partitions: 4

Island iteration time: 3.0848997994326055e-05s

Best chromosome ->: [(46, (array([ 1.72000213,  0.81874225,  1.48003564,  3.49314839]), 43.227477169653518))]

Migration number: 6

Number of Evaluations 1040
Number of partitions: 4

Island iteration time: 2.8873000701423734e-05s

Best chromosome ->: [(52, (array([ 1.63113886,  0.5240396 ,  2.08886851,  3.49814407]), 39.55047486185348))]

Migration number: 7

Number of Evaluations 1200
Number of partitions: 4

Island iteration time: 1.4436998753808439e-05s

Best chromosome ->: [(65, (array([ 1.63334041,  0.46690354,  2.12655854,  3.49340661]), 39.290553757755006))]

Migration number: 8

Number of Evaluations 1360
Number of partitions: 4

Island iteration time: 2.6025998522527516e-05s

Best chromosome ->: [(73, (array([ 1.63334041,  0.46690354,  2.12655854,  


Best chromosome ->: [(70, (array([ 0.14221715,  3.15885342, -4.93372353,  3.33680079]), 52.165122013301755))]

Migration number: 1

Number of Evaluations 240
Number of partitions: 4

Island iteration time: 2.716099697863683e-05s

Best chromosome ->: [(26, (array([ 0.14221715,  3.15885342, -4.93372353,  3.33680079]), 52.165122013301755))]

Migration number: 2

Number of Evaluations 400
Number of partitions: 4

Island iteration time: 2.8440997994039208e-05s

Best chromosome ->: [(74, (array([ 1.0362629 ,  0.96855118, -4.24857594,  2.94677348]), 36.377582430970797))]

Migration number: 3

Number of Evaluations 560
Number of partitions: 4

Island iteration time: 3.571599518181756e-05s

Best chromosome ->: [(2, (array([ 0.0804686 ,  1.83366725, -4.6565309 ,  4.22220425]), 34.157599010099247))]

Migration number: 4

Number of Evaluations 720
Number of partitions: 4

Island iteration time: 3.190699499100447e-05s

Best chromosome ->: [(0, (array([ 0.0804686 ,  1.83366725, -4.6565309 ,  4.2222


Best chromosome ->: [(65, (array([ 0.32119288, -2.55766324,  0.91389641,  4.04866804]), -993.04838991941358))]

Migration number: 1

Number of Evaluations 240
Number of partitions: 4

Island iteration time: 2.78999941656366e-05s

Best chromosome ->: [(37, (array([-0.97006241, -2.21015071,  0.59083074,  3.22096415]), -993.12532732538943))]

Migration number: 2

Number of Evaluations 400
Number of partitions: 4

Island iteration time: 2.859200321836397e-05s

Best chromosome ->: [(63, (array([-0.97006241, -2.21015071,  0.59083074,  3.22096415]), -993.12532732538943))]

Migration number: 3

Number of Evaluations 560
Number of partitions: 4

Island iteration time: 2.8804002795368433e-05s

Best chromosome ->: [(68, (array([-1.36663061, -2.27800703,  1.35119042,  3.92655479]), -994.5058490288078))]

Migration number: 4

Number of Evaluations 720
Number of partitions: 4

Island iteration time: 2.938599936896935e-05s

Best chromosome ->: [(17, (array([-1.36663061, -2.27800703,  1.35119042,  3.


Best chromosome ->: [(16, (array([-3.7902697 ,  2.65272469, -3.67951882, -0.50498343]), 2659.9140753488277))]

Migration number: 1

Number of Evaluations 240
Number of partitions: 4

Island iteration time: 1.3030999980401248e-05s

Best chromosome ->: [(55, (array([-4.94548079,  2.48313496, -5.14645817,  1.51724951]), 1022.9693316161347))]

Migration number: 2

Number of Evaluations 400
Number of partitions: 4

Island iteration time: 2.6148998585995287e-05s

Best chromosome ->: [(17, (array([-4.94548079,  2.48313496, -5.14645817,  1.51724951]), 1022.9693316161347))]

Migration number: 3

Number of Evaluations 560
Number of partitions: 4

Island iteration time: 2.7528003556653857e-05s

Best chromosome ->: [(9, (array([-4.12204748,  1.89966114, -3.73159897, -0.32651399]), 132.0705495717342))]

Migration number: 4

Number of Evaluations 720
Number of partitions: 4

Island iteration time: 1.001299824565649e-05s

Best chromosome ->: [(53, (array([-4.12204748,  1.89966114, -3.73159897, -0.32

...


Best chromosome ->: [(69, (array([-1.83412616, -3.51828994,  0.97147215,  3.3298639 ]), -958.29777612965029))]

Migration number: 1

Number of Evaluations 240
Number of partitions: 4

Island iteration time: 3.972400008933619e-05s

Best chromosome ->: [(46, (array([-0.24249064, -0.90943693, -1.94404191,  4.14683551]), -973.92200305261395))]

Migration number: 2

Number of Evaluations 400
Number of partitions: 4

Island iteration time: 1.3220000255387276e-05s

Best chromosome ->: [(8, (array([-2.2989993 , -1.97462371,  0.33222241,  4.207486  ]), -987.32648661525491))]

Migration number: 3

Number of Evaluations 560
Number of partitions: 4

Island iteration time: 4.319399886298925e-05s

Best chromosome ->: [(34, (array([-1.45064323, -1.29119884,  0.04234355,  4.02160189]), -987.659860059067))]

Migration number: 4

Number of Evaluations 720
Number of partitions: 4

Island iteration time: 2.4163004127331078e-05s

Best chromosome ->: [(18, (array([-2.1204491 , -2.94091941,  0.97147215,  4.


Best chromosome ->: [(57, (array([-4.44332626,  1.4358462 , -4.52238615,  1.93517787]), 144.93046992776848))]

Migration number: 1

Number of Evaluations 240
Number of partitions: 4

Island iteration time: 2.849999873433262e-05s

Best chromosome ->: [(57, (array([-4.44332626,  1.4358462 , -4.52238615,  1.93517787]), 144.93046992776848))]

Migration number: 2

Number of Evaluations 400
Number of partitions: 4

Island iteration time: 1.783000334398821e-05s

Best chromosome ->: [(3, (array([-4.44332626,  1.4358462 , -4.52238615,  1.93517787]), 144.93046992776848))]

Migration number: 3

Number of Evaluations 560
Number of partitions: 4

Island iteration time: 2.7677000616677105e-05s

Best chromosome ->: [(10, (array([-4.44332626,  1.4358462 , -4.52238615,  1.93517787]), 144.93046992776848))]

Migration number: 4

Number of Evaluations 720
Number of partitions: 4

Island iteration time: 1.3037999451626092e-05s

Best chromosome ->: [(8, (array([-4.44332626,  1.4358462 , -4.52238615,  1.935


Best chromosome ->: [(16, (array([ 3.63223401, -2.37392954, -3.4835454 ,  0.94113235]), 611.04538251751092))]

Migration number: 1

Number of Evaluations 240
Number of partitions: 4

Island iteration time: 2.625000342959538e-05s

Best chromosome ->: [(72, (array([ 2.50546879, -3.0985285 , -3.33915027, -0.21582015]), 596.56915003121117))]

Migration number: 2

Number of Evaluations 400
Number of partitions: 4

Island iteration time: 4.0734994399826974e-05s

Best chromosome ->: [(69, (array([ 2.50546879, -3.0985285 , -3.33915027, -0.21582015]), 596.56915003121117))]

Migration number: 3

Number of Evaluations 560
Number of partitions: 4

Island iteration time: 1.5559999155811965e-05s

Best chromosome ->: [(51, (array([ 2.50546879, -3.0985285 , -3.33915027, -0.21582015]), 596.56915003121117))]

Migration number: 4

Number of Evaluations 720
Number of partitions: 4

Island iteration time: 4.349200025899336e-05s

Best chromosome ->: [(33, (array([ 2.50546879, -3.0985285 , -3.33915027, -0.2


Best chromosome ->: [(50, (array([ 3.12587683, -0.45464694, -4.5646079 , -3.89788038]), 41.683952158151357))]

Migration number: 1

Number of Evaluations 240
Number of partitions: 4

Island iteration time: 2.8688002203125507e-05s

Best chromosome ->: [(6, (array([ 3.12587683, -0.45464694, -4.5646079 , -3.89788038]), 41.683952158151357))]

Migration number: 2

Number of Evaluations 400
Number of partitions: 4

Island iteration time: 4.639800317818299e-05s

Best chromosome ->: [(47, (array([ 3.12587683, -0.45464694, -4.5646079 , -3.89788038]), 41.683952158151357))]

Migration number: 3

Number of Evaluations 560
Number of partitions: 4

Island iteration time: 3.2403993827756494e-05s

Best chromosome ->: [(4, (array([ 3.12587683, -0.45464694, -4.5646079 , -3.89788038]), 41.683952158151357))]

Migration number: 4

Number of Evaluations 720
Number of partitions: 4

Island iteration time: 2.6073998014908284e-05s

Best chromosome ->: [(12, (array([ 3.21725992, -1.61151819, -3.55726693, -3.60


Best chromosome ->: [(53, (array([-2.75160946, -4.7590592 , -3.4654821 , -0.16038746]), -112.30493811242206))]

Migration number: 1

Number of Evaluations 240
Number of partitions: 4

Island iteration time: 1.284600148210302e-05s

Best chromosome ->: [(36, (array([-3.37278847, -3.35641161, -4.30719269,  1.0597061 ]), -126.42646758159729))]

Migration number: 2

Number of Evaluations 400
Number of partitions: 4

Island iteration time: 1.2368000170681626e-05s

Best chromosome ->: [(0, (array([-1.4575243 , -3.13995089, -3.76383251, -0.57448567]), -130.961809160288))]

Migration number: 3

Number of Evaluations 560
Number of partitions: 4

Island iteration time: 2.6394001906737685e-05s

Best chromosome ->: [(31, (array([-1.4575243 , -3.13995089, -3.76383251, -0.57448567]), -130.961809160288))]

Migration number: 4

Number of Evaluations 720
Number of partitions: 4

Island iteration time: 2.9217997507657856e-05s

Best chromosome ->: [(51, (array([-1.4575243 , -3.13995089, -3.76383251, -0.5